In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas.tseries import offsets
from skforecast import __version__
print(__version__)


0.10.1


In [3]:
# simulate a time series
n = 365
y = pd.Series(
    data=np.arange(n),
    index = pd.date_range(start='2000-01-01', periods=n, freq='d')
)
y

2000-01-01      0
2000-01-02      1
2000-01-03      2
2000-01-04      3
2000-01-05      4
             ... 
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Length: 365, dtype: int32

In [4]:
forecaster = ForecasterEquivalentDate(offset=10, n_offsets=2)
forecaster.fit(y=y)
forecaster

ForecasterEquivalentDate 
Offset: 10 
Number of offsets: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-11-11 19:05:21 
Last fit date: 2023-11-11 19:05:21 
Skforecast version: 0.10.1 
Python version: 3.10.11 
Forecaster id: None 

In [5]:
forecaster.predict(steps=15)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
Freq: D, Name: pred, dtype: float64

In [6]:
metric, predictions_1 = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=20,
    steps=15,
    metric='mean_absolute_error',
    verbose=False
)

  0%|          | 0/23 [00:00<?, ?it/s]

In [86]:
forecaster = ForecasterEquivalentDate(
    offset=offsets.Day(10),
    n_offsets=2
)
forecaster.fit(y=y)
forecaster

ForecasterEquivalentDate 
Offset: <10 * Days> 
Number of offsets: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-11-06 13:38:03 
Last fit date: 2023-11-06 13:38:03 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [87]:
forecaster.predict(steps=15)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
Freq: D, Name: pred, dtype: float64

In [10]:
forecaster = ForecasterEquivalentDate(
    offset = offsets.Day(10),
    n_offsets=2
)

print(forecaster.regressor)

metric, predictions_2 = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=20,
    steps=15,
    metric='mean_absolute_error',
    refit=True,
    verbose=False,
    n_jobs='auto'
)
predictions_2

None


  0%|          | 0/23 [00:00<?, ?it/s]

,pred
2000-01-21,5.0
2000-01-22,6.0
2000-01-23,7.0
2000-01-24,8.0
2000-01-25,9.0
...,...
2000-12-26,335.0
2000-12-27,336.0
2000-12-28,337.0
2000-12-29,338.0


In [89]:
pd.testing.assert_frame_equal(predictions_1, predictions_2)